### Importing the dependencies to work with

In [2]:
import lmfit
import rampy as rp #Charles' libraries and functions

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px


###  Reading the data file

In [3]:
# Load Data
import os
dir_name = 'Data/Thylakoids/qg27/'
filename = 'qg27_thyl_XY09'
full_file = os.path.join(dir_name,filename)
data = pd.read_csv(full_file, delimiter = "\t", names=["x", "y"])



### Visualize the Data

In [4]:
fig = px.line(data_frame=data, x=data['x'], y=data['y'])
fig.show()

In [ ]:
# Convert Dataframe data to numpy arrays for working with smoothening and correction
data_array = data.to_numpy()
x = data_array[:,0]
y = data_array[:,1]


In [ ]:
data_array = all_data.to_numpy()
data_array
x_array = all_data.index.to_numpy()
print(data_array)
print(x_array)
print(np.ndim(data_array))
# np.savetxt("all_y.csv",data_array, delimiter=",")

In [ ]:
x = x_array
y = data_array[:,25]
print(y)

For smoothening

In [ ]:
# Incase to smooth
# y_smo = rp.smooth(data_array[:,0],data_array[:,1],method="savgol",window_length=5,polyorder=3)
# y_smo = rp.smooth(x,y,method="whittaker",Lambda=10**0.5)

Check the smoothen and raw data

In [ ]:
# Check the plots before and after smooth
# plt.figure(figsize=(8,8))
# plt.subplot(2,1,1)
# plt.plot(x, y)
# plt.title("Raw Data")
# plt.subplot(2,1,2)
# plt.title("Smoothen Data")
# plt.plot(data_array[:,0], y_smo)

#### Select the region of Interest (ROI)

In [ ]:
roi = np.array([(1347,1365),(1774,1800)])
roi[1,1]

### Baseline Fitting and trimming the data

In [ ]:
# Base line correction
y_corr, y_base = rp.baseline(x,y,roi,'poly',polynomial_order=3)

# Trim the data
x_fit = x[np.where((x > roi[0,0])&(x < roi[1,1]))]
y_fit = y_corr[np.where((x > roi[0,0])&(x < roi[1,1]))]

Plot the trimmed data

In [ ]:
plt.figure()
plt.plot(x_fit,y_fit)
# plt.xlim([1347,1849])
# plt.title("Trimmed Data non-normalized")

Normalized the trimmed data and plot

In [ ]:
y_fit_norm_intensity = rp.normalise(y_fit,x=x_fit,method="intensity")
plt.plot(x_fit,y_fit_norm_intensity)
# plt.title("Trimmed Data normalized to max intensity")


In [ ]:
# create a new plot for showing the spectrum
plt.figure(dpi=150)
plt.subplot(1,2,1)
inp = plt.plot(x,y,'k-',label='Original')
corr = plt.plot(x,y_corr,'b-',label='Corrected') #we use the sample variable because it is not already normalized...
bas = plt.plot(x,y_base,'r-',label='Baseline')
plt.xlim(roi[0,0],roi[1,1])
plt.ylim(-0.1,2)
plt.xlabel("Wavelength, cm$^{-1}$", fontsize = 14)
plt.ylabel("Normalized intensity, a. u.", fontsize = 14)
plt.legend()
plt.title('A) Baseline removal')

plt.subplot(1,2,2)
plt.plot(x_fit,y_fit_norm_intensity,'k.')
plt.xlabel("Wavelength, cm$^{-1}$", fontsize = 14)
plt.title('B) signal to fit')
#plt.tight_layout()
# plt.suptitle('Figure 2', fontsize = 14,fontweight = 'bold')

### Replot for quick check for the input paramaters 

In [ ]:
arr = np.column_stack((x_fit,y_fit_norm_intensity))
my_array = pd.DataFrame(arr, columns = ['x_fit','y_fit'])
fig = px.line(data_frame=my_array, x=my_array['x_fit'], y=my_array['y_fit'])
fig['layout']['xaxis']['autorange'] = "reversed"
fig.show()

It would be good to check the plot and assign initial parameters in the parameters section for good and faster fit.

### Parameters Section

Make sure to add equal number of parameters as you have used in case of the residual local function . 

In [ ]:
params = lmfit.Parameters()
algo = 'leastsq'
#               (Name,  Value,   Vary,   Min,     Max,       Expr)
params.add_many(
                # Protein
                ('a1',   0.1,    True,    0,      0.2,     None), #Amplitude
                ('f1',   1740,   True,   1735,    1755,    None), #position
                ('l1',   5,      True,    1,      50,     None), # FWHM

                ('a2',   0.04,    True,     0,      0.2,    None),   #Amplitude
                ('f2',   1725,   True,   1720,     1745,     None),
                ('l2',    5,    True,    1,       50,    None), 
                # Lipid
                ('a3',   0.9,    True,    0,       1,        None),  # Amplitude
                ('f3',   1654,   True,    1640,   1660,     None),
                ('l3',   20,     True,     10,     40,      None),

                ('a4',   0.9,    True,      0,      1,      None), # Amplitude
                ('f4',   1636,   True,   1630,    1650,     None),
                ('l4',   20,     True,     10,     80,      None), 
                
                ####
                ('a5',   0.3,    True,    0.0,     1,       None), # Amplitude
                ('f5',   1547,   True,    1592,    1505,    None),
                ('l5',   40,     True,     0,      100,     None),

                ('a6',   0.7,    True,     0.0,       1,    None), # Amplitude
                ('f6',   1453,   True,    1425,    1470,    None),
                ('l6',   20,     True,     0,      50,      None),   
                
                ('a7',   0.07,    True,     0.0,    1,      None), # Amplitude
                ('f7',   1378,   True,    1350,    1380,    None),
                ('l7',   10,     True,     0,      10,      None),   
                        
               )
params  

### Run Fit

In [ ]:
from Modules.custom_funx import*


In [ ]:
result = lmfit.minimize(residual, params, method = algo, args=(x_fit, y_fit_norm_intensity[:,0]))
print(lmfit.fit_report(result))

### Get all the residuals

In [ ]:
yout, peak1,peak2,peak3,peak4,peak5,peak6, peak7 = residual(result.params,x_fit)
res_stack = np.column_stack((yout,peak1,peak2,peak3,peak4,peak5,peak6,peak7))



In [ ]:
# Create dataframe
res_df = create_residuals_df(res_stack,y_fit_norm_intensity)
# Create Plots
ax = plot_fit(res_df,x_fit)
r = cal_peak(peak1,peak2, peak3, peak4, x_fit)
ax.title( "Protein/lipid ratio: "+str(round(r,2)))
# pip install pyppt
import pyppt as ppt
ppt.add_figure('Left',delete_placeholders=True, replace=True, slide_no=2)
ppt.set_title(filename, slide_no=1)
ppt.title_to_front(slide_no=2)
ax1 = plot_res(x_fit,result.residual)
ax1.title('Reduced ChiSq - '+str(round(result.redchi,5)))
ppt.add_figure('Right', delete_placeholders=True, replace=True)




In [ ]:
res = y_fit_norm_intensity - yout
np.savetxt("y_fit_norm_intensity.csv",y_fit_norm_intensity, delimiter=",")
np.savetxt("yout.csv",yout, delimiter=",")
np.savetxt("res.csv",res, delimiter=",")
np.savetxt("x_fit.csv",x_fit, delimiter=",")